In [1]:
from pygooglenews import GoogleNews
import pandas as pd
import PySimpleGUI as sg
import datetime
import base64

In [6]:
gn = GoogleNews(lang='EG', country='MY')

def gui():
    websites = []
    excludes= []
    phrases = []
    sg.ChangeLookAndFeel('Topanga')
    #sg.set_options(icon=base64.b64encode(open(r'./image/test.png', 'rb').read()))

    scrape_button = 'Start Scrape'
    layout = [
        [sg.Text('Google News Scraping',font=("Lucida", 15, 'bold'), justification='c',text_color='white')],
        [sg.Text(' ', size=(11, 1))],
        [sg.Text('Website:', size=(11, 1)), sg.InputText()],
        [sg.Text('Insert "; " to add more website. Ex: web.com; webs.org')],
        [sg.Text(' ', size=(11, 1))],
        [sg.Text('Exact Phrase:', size=(11, 1)), sg.InputText()],
        [sg.Text(' ', size=(11, 1))],
        [sg.Text('Has Word:',size = (11,1)),sg.InputText()],
        [sg.Text(' ', size=(11, 1))],
        [sg.Text('Exclude:', size=(11, 1)), sg.InputText()],
        [sg.Text('Insert "-" to exclude more word. Ex: word1 -word2')],
        [sg.Text(' ', size=(11, 1))],
        [sg.Text('Start Date:', size=(11, 1)), sg.InputText(key='input1')],
        [sg.CalendarButton('Start Date', target='input1', key='date1', format='%d/%m/%Y')],
        [sg.Text('End Date:', size=(11, 1)), sg.InputText(key='input2')],
        [sg.CalendarButton('End date', target='input2', key='date2', format='%d/%m/%Y')],
        [sg.Text(' ', size=(11, 1))],
        [sg.Text('File Name:', size=(11, 1)), sg.InputText()],
        [sg.Text(' ', size=(25, 1), key='-STATUS-')],
        [sg.Button(scrape_button, button_color=('white', 'brown'))]
         ]
    window = sg.Window(title = 'Scrape Google News', layout = layout, font=("Helvetica", 11), resizable=False)

    while True:
        event, values = window.read()
        if event == sg.WIN_CLOSED:
            break
        if event == scrape_button:
            stories = []
            websites = values[0].split("; ")    #Can be adjusted
            phrases = values[1]
            excludes = values[3]

            if not websites:
                compile = values[2] + ' "' + phases + '"' + " -" + excludes
                stories = get_title(compile,values['input1'],values['input2'])

            else:
                for website in websites:
                    compile = values[2] + " site:" + website + ' "' + phases + '"' +" -" + excludes
                    story = get_title(compile, values['input1'], values['input2'])
                    stories += story

                    #print("code already through")
                    #print(website)
                    #print(compile)

            if not stories:
                window.Element('-STATUS-').update("Fail :(")
            else:
                result = file_name(values[4], stories)
                window.Element('-STATUS-').update(result)

    window.close()

In [7]:
def get_title(search, date_from, date_to):
    stories = []
    try:
        to_date = date_to.split('/')
        from_date = date_from.split('/')
        start_date = datetime.date(int(from_date[2]), int(from_date[1]),
                                   int(from_date[0]))
        end_date = datetime.date(int(to_date[2]), int(to_date[1]),
                                 int(to_date[0]))

    except:
        return stories

    search = gn.search(search, from_= start_date.strftime('%Y-%m-%d') , to_= end_date.strftime('%Y-%m-%d') )
    newsitem = search['entries']

    for item in newsitem:
        time = item.published.split(', ')[1]
        time = time.split(' ')[0] +' '+  time.split(' ')[1] + ' ' + time.split(' ')[2]

        story ={
            'title' : item.summary.split('">')[1].split('</a>')[0],
            'published': time,
            'source': item.source.title,
            'link' : item.link
        }
        stories.append(story)
    #print("Hi")
    #print(stories)
    return stories

In [ ]:
def file_name(export,stories):
    try:
        df = pd.DataFrame(stories)
        file = r'./' + export + '.csv'
        df.to_csv(file, index=False)
        return 'Success :D'

    except:
        return 'Error: Unable to save file'



gui()